# CFC Risk Detector

#### Info
Developed as part of the Deloitte Tech Startup Challenge.
Essentially has two componentes, simulated subsidiary generator and CFC compliance detector.
The attributes of the simulated subsidiary generated are quasi arbitrary but set to meet the thresholds of the regulation itself.
The CFC complicance detector also returns the amount of extra tax that would be paid in a post CFC circumstance.
Our system only returns Option A CFC outcomes,  and as such operates on a subsidiary by subsidiary basis under the assumption that all subsiaries are 100% owned by their parent companies.
### Future Directions
CFC Option B outcomes require analyzing the larger org-chart of the corporation represented as a graph. Due to the time constraints of the challenge this was not performed. However, the following tool could still be used in such a case in which a search is conducted over the org-chart represented as a graph, whose nodes are subsidiaries and edges are % ownership with minor, 





In [221]:
#Globals
tax_rates = {'NL': .25,
            'CM' : .01}

ETR_t = .125
SAE_t = 5
income_t = .33





class Subsidiary():
    
    def __init__(self, shares, capital, profit, earnings, activity, home, local, earnings_f, ETR):
        self.shares = shares
        self.home = home
        self.capital = capital
        self.profit = profit
        self.earnings = earnings
        self.activity = activity
        self.CFC_violation = False
        self.CFC = None
        self.local = local
        self.pre = earnings_f*tax_rates[self.local]
        self.post = self.pre #add pre and post CFC
        self.delta = 0 
        self.earnings_f = earnings_f
        self.ETR = ETR
        

#test = Subsidiary(shares, capital, profit, earnings, activity, home, local, earnings_f)

        



In [1]:
import numpy as np


def gen_subsidiary():
    #draw type from multinomial
    i = np.random.multinomial(1,[.90,.06,.04], size = 1)
    i = np.where(i == 1)[1][0]
    
    
    #if non CFC
    if i == 0:
        
        
        
        
        shares = np.random.normal(.6,.1)
        capital = np.random.normal(.3,.05)
        profit = np.random.normal(.25,.07)
        ETR = .25
        activity = np.random.normal(20,2)
        
        
        earnings = .50
        earnings_f = np.random.normal(100000,10000)
        home = 'NL'
        local = 'CM'
        
        
    #if CFC
    elif i == 1:
        
        shares = np.random.normal(.6,.1)
        capital = np.random.normal(.6,.05)
        profit = np.random.normal(.6,.07)
        ETR = .10
        activity = np.random.normal(20,2)
        
        
        earnings = .25
        earnings_f = np.random.normal(100000,10000)
        home = 'NL'
        local = 'CM'
        
    #if CFCviolator
    else:
        
        shares = np.random.normal(.6,.1)
        capital = np.random.normal(.6,.05)
        profit = np.random.normal(.6,.07)
        ETR = np.random.normal(.10,.02)
        activity = np.random.normal(3,2)
        
        
        earnings = .50
        earnings_f = np.random.normal(100000,10000)
        home = 'NL'
        local = 'CM'
        
    
    
    subsidiary = Subsidiary(shares, capital, profit, earnings, activity, home, local, earnings_f, ETR)

    return subsidiary
    
    

    

#### Generate Simulated Subsidiary Data

In [2]:
import os

size = 100
output = np.zeros(shape = (size, 4))


for i in range(size):
    output[i,:] = decision_procedure(gen_subsidiary())

np.savetxt(os.getcwd()+'/output.csv', output, delimiter=',')



NameError: name 'decision_procedure' is not defined

#### Check CFC Compliance

In [224]:
def CFC_checker(subsidiary):
    '''
    Function to check if CFC criteria met
    #check if shares > 50%
    #check if capital > 50%
    #check if profit > 50%
    '''
    CFC = subsidiary.shares > .5 or subsidiary.capital > .5 or subsidiary.profit > .5 and ETR

    CFC = CFC and subsidiary.ETR < ETR_t

    subsidiary.CFC = CFC

    #if all three are met subsidiary is CFC
    return CFC

def CFC_violation(subsidiary):
    
    '''
    checks if the following criteria are met
    Subsantial economic activity > threshold
    Income by category > threshold
    '''
    
    subsidiary.CFC_violation = activity > SAE_t and subsidiary.earnings > income_t
    return subsidiary.CFC_violation

def severity_check(subsidiary):
    '''
    Given a CFC violating company
    whats the amount of tax pre and post CFC
    '''
    #only tax paid on passive income
    
    subsidiary.post = subsidiary.earnings_f*tax_rates[subsidiary.home]
    subsidiary.delta = subsidiary.post - subsidiary.pre
    
def decision_procedure(subsidiary):
    
    #first check if CFC
    if CFC_checker(subsidiary):
        
        #then check violation
        if CFC_violation(subsidiary):
            
            #then get the difference in tax payed
            severity_check(subsidiary)
        else:
            pass
        
            
    else:
        pass
    
    return (subsidiary.CFC, subsidiary.CFC_violation,
           subsidiary.pre, subsidiary.post)
        
    
    
    
    
    
    
    

    

CFC Violator


(True, True, 1000.0, 25000.0)

Vertices of graph:
['a', 'b', 'c', 'd', 'e', 'f']
Edges of graph:
[{'a', 'd'}, {'b', 'c'}, {'c'}, {'d', 'c'}, {'e', 'c'}]
Add vertex:
Vertices of graph:
['a', 'b', 'c', 'd', 'e', 'f', 'z']
Add an edge:
Vertices of graph:
['a', 'b', 'c', 'd', 'e', 'f', 'z']
Edges of graph:
[{'a', 'd'}, {'a', 'z'}, {'b', 'c'}, {'c'}, {'d', 'c'}, {'e', 'c'}]
Adding an edge {"x","y"} with new vertices:
Vertices of graph:
['a', 'b', 'c', 'd', 'e', 'f', 'z', 'y']
Edges of graph:
[{'a', 'd'}, {'a', 'z'}, {'b', 'c'}, {'c'}, {'d', 'c'}, {'e', 'c'}, {'y', 'x'}]


first
second
0
second
1
second
2
first
second
0
second
1
second
2
